In [1]:
import warnings
warnings.filterwarnings('ignore') 

try:
    %tensorflow_version 2.x
except Exception:
    pass

import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

In [2]:
label = ['PNEUMONIA', 'NORMAL']
img_size = 150

train_image_path = 'D:/SideProject/kaggle/dataset/chest X-Ray/chest_xray/train'
train_NORMAL_path = os.path.join(train_image_path, 'NORMAL')
train_PNEUMONIA_path = os.path.join(train_image_path, 'PNEUMONIA')

test_image_path = 'D:/SideProject/kaggle/dataset/chest X-Ray/chest_xray/test'
test_NORMAL_path = os.path.join(test_image_path, 'NORMAL')
test_PNEUMONIA_path = os.path.join(test_image_path, 'PNEUMONIA')

val_image_path = 'D:/SideProject/kaggle/dataset/chest X-Ray/chest_xray/val'
val_NORMAL_path = os.path.join(val_image_path, 'NORMAL')
val_PNEUMONIA_path = os.path.join(val_image_path, 'PNEUMONIA')

In [3]:
train_file_list_NORMAL = os.listdir(train_NORMAL_path)
train_file_list_PNEUMONIA = os.listdir(train_PNEUMONIA_path)
print(len(train_file_list_NORMAL) + len(train_file_list_PNEUMONIA))

test_file_list_NORMAL = os.listdir(test_NORMAL_path)
test_file_list_PNEUMONIA = os.listdir(test_PNEUMONIA_path)
print(len(test_file_list_NORMAL) + len(test_file_list_PNEUMONIA))

val_file_list_NORMAL = os.listdir(val_NORMAL_path)
val_file_list_PNEUMONIA = os.listdir(val_PNEUMONIA_path)
print(len(val_file_list_NORMAL) + len(val_file_list_PNEUMONIA))

5216
624
16


In [4]:
traincnt = 5126
testcnt = 624
batchsz = 20

train_datagen = ImageDataGenerator(rescale=1./255, zoom_range=0.2, shear_range=0.2, horizontal_flip=True)

train_generator = train_datagen.flow_from_directory(train_image_path, target_size=(64, 64), batch_size=batchsz)

test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(test_image_path, target_size=(64, 64), batch_size=batchsz)

val_datagen = ImageDataGenerator(rescale=1./255)
val_generator = val_datagen.flow_from_directory(val_image_path, target_size=(64, 64), batch_size=batchsz)

Found 5216 images belonging to 2 classes.
Found 624 images belonging to 2 classes.
Found 16 images belonging to 2 classes.


In [5]:
epoch_param = 50
print("[INFO] compiling model...")

model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(input_shape=(64, 64, 3), kernel_size=(3,3), filters=32, activation='relu', padding='same'),
    tf.keras.layers.MaxPool2D(strides=(2,2)),
    tf.keras.layers.Dropout(rate=0.2),
    tf.keras.layers.Conv2D(kernel_size=(3,3), filters=64, activation='relu', padding='same'),
    tf.keras.layers.MaxPool2D(strides=(2,2)),
    tf.keras.layers.Dropout(rate=0.2),
    tf.keras.layers.Conv2D(kernel_size=(3,3), filters=128, activation='relu', padding='same'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units=128, activation='relu'),
    tf.keras.layers.Dropout(rate=0.5),
    tf.keras.layers.Dense(units=64, activation='relu'),
    tf.keras.layers.Dropout(rate=0.5),
    tf.keras.layers.Dense(units=2, activation='softmax')
])
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()
history = model.fit(train_generator, steps_per_epoch=(traincnt//batchsz), validation_data=(val_generator),epochs=epoch_param)

[INFO] compiling model...
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 64, 64, 32)        896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 32, 32, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 16, 16, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (

Epoch 43/50
256/256 [==============================] - 50s 197ms/step - loss: 0.1120 - accuracy: 0.9617 - val_loss: 0.4005 - val_accuracy: 0.8750
Epoch 44/50
256/256 [==============================] - 50s 197ms/step - loss: 0.1046 - accuracy: 0.9590 - val_loss: 0.3525 - val_accuracy: 0.8125
Epoch 45/50
256/256 [==============================] - 50s 196ms/step - loss: 0.1072 - accuracy: 0.9619 - val_loss: 0.2685 - val_accuracy: 0.9375
Epoch 46/50
256/256 [==============================] - 51s 197ms/step - loss: 0.1021 - accuracy: 0.9672 - val_loss: 0.6387 - val_accuracy: 0.7500
Epoch 47/50
256/256 [==============================] - 50s 197ms/step - loss: 0.0879 - accuracy: 0.9684 - val_loss: 0.4591 - val_accuracy: 0.8125
Epoch 48/50
256/256 [==============================] - 50s 197ms/step - loss: 0.1032 - accuracy: 0.9617 - val_loss: 0.3420 - val_accuracy: 0.7500
Epoch 49/50
256/256 [==============================] - 50s 196ms/step - loss: 0.1099 - accuracy: 0.9633 - val_loss: 0.5984 -

In [6]:
model.save('classification_pneumnia_model_add_val.h5')

model.evaluate(test_generator, verbose=0)

[0.5223275423049927, 0.9102563858032227]